# Pandas Data "Munging"

- Merging, joining, and concatenating
- Aggregation and grouping

# Merging data

Sometimes our data is spread over a bunch of different `DataFrame`s. To use it with Scikit-Learn, we'll need to create a *single* feature matrix. For this, we can `merge` the `DataFrame`s (similar to a SQL JOIN):

In [1]:
import pandas as pd
product = pd.read_csv('../data/prices.csv')
product.head()

,sku,category,price
0,sku0000,Toilets,2085.11
1,sku0001,Misc,3601.62
2,sku0002,Misc,0.57
3,sku0003,Faucets,1511.66
4,sku0004,Toilets,733.78


In [2]:
sales = pd.read_csv('../data/sales.csv', parse_dates=['date'])
sales.head()

,order_num,line_num,date,sku,qty
0,0,0,2011-01-01,sku4333,6
1,0,1,2011-01-01,sku76536,7
2,1,0,2011-01-02,sku75108,3
3,1,1,2011-01-02,sku78838,9
4,1,2,2011-01-02,sku77480,9


These two `DataFrame`s have a common column "sku".  A "quick-and-dirty" solution is to let Pandas merge on common column names (the default):

In [3]:
pd.merge(sales, product).head()

,order_num,line_num,date,sku,qty,category,price
0,0,0,2011-01-01,sku4333,6,Showers,1563.90
1,641,3,2012-10-10,sku4333,8,Showers,1563.90
2,0,1,2011-01-01,sku76536,7,Faucets,3121.99
3,1,0,2011-01-02,sku75108,3,Misc,819.55
4,1,1,2011-01-02,sku78838,9,Showers,2531.54


If you want more control, you can specify *which* column to merge on, as well as *what kind* of join you want to use (inner, left, right, or outer):

In [4]:
merged = pd.merge(sales, product, on='sku', how='left')
merged.head()

,order_num,line_num,date,sku,qty,category,price
0,0,0,2011-01-01,sku4333,6,Showers,1563.90
1,0,1,2011-01-01,sku76536,7,Faucets,3121.99
2,1,0,2011-01-02,sku75108,3,Misc,819.55
3,1,1,2011-01-02,sku78838,9,Showers,2531.54
4,1,2,2011-01-02,sku77480,9,Faucets,848.53


If you want to merge based on an index value, you can specify to use the index:

In [5]:
iproduct = product.set_index('sku')

In [6]:
iproduct.head()

,category,price
sku,,
sku0000,Toilets,2085.11
sku0001,Misc,3601.62
sku0002,Misc,0.57
sku0003,Faucets,1511.66
sku0004,Toilets,733.78


In [7]:
pd.merge(sales, iproduct, left_on='sku', right_index=True).head()

,order_num,line_num,date,sku,qty,category,price
0,0,0,2011-01-01,sku4333,6,Showers,1563.90
1601,641,3,2012-10-10,sku4333,8,Showers,1563.90
1,0,1,2011-01-01,sku76536,7,Faucets,3121.99
2,1,0,2011-01-02,sku75108,3,Misc,819.55
3,1,1,2011-01-02,sku78838,9,Showers,2531.54


You can also use the `.join` method on `DataFrame`s to do th same thing:

In [8]:
sales.join(iproduct, on='sku').head()

,order_num,line_num,date,sku,qty,category,price
0,0,0,2011-01-01,sku4333,6,Showers,1563.90
1,0,1,2011-01-01,sku76536,7,Faucets,3121.99
2,1,0,2011-01-02,sku75108,3,Misc,819.55
3,1,1,2011-01-02,sku78838,9,Showers,2531.54
4,1,2,2011-01-02,sku77480,9,Faucets,848.53


# Grouping data

Sometimes the data we want is at the wrong level of aggregation (e.g. we want weekly numbers, but we're given daily numbers, or we need regional and we're given per-store).

In this case, we might want to get monthly sales. To do this, we first need the total price per line item (the "extended" price) and the month name:

In [9]:
merged['extended'] = merged.qty * merged.price
merged['month_name'] = merged.date.dt.month_name()
merged.head()

,order_num,line_num,date,sku,qty,category,price,extended,month_name
0,0,0,2011-01-01,sku4333,6,Showers,1563.90,9383.40,January
1,0,1,2011-01-01,sku76536,7,Faucets,3121.99,21853.93,January
2,1,0,2011-01-02,sku75108,3,Misc,819.55,2458.65,January
3,1,1,2011-01-02,sku78838,9,Showers,2531.54,22783.86,January
4,1,2,2011-01-02,sku77480,9,Faucets,848.53,7636.77,January


Now, we can group by the month name and sum the extended price:

In [10]:
merged.groupby('month_name').extended.sum()

month_name
April        2420895.85
August       2501007.56
December     2081972.98
February     2089458.92
January      2389470.69
July         2474997.68
June         2946294.24
March        3212641.31
May          3250057.87
November     2009360.01
October      2001403.95
September    2515467.17
Name: extended, dtype: float64

We can do other groupings just as easily. For category and month:

In [11]:
g = merged.groupby(['category', 'month_name'])

In [12]:
g

In [13]:
g.extended.mean()

category  month_name
Faucets   April         10882.773878
          August        10552.072982
          December       9691.680465
          February       9677.353590
          January       13512.062745
          July          13370.439697
          June           9540.325273
          March         14321.106667
          May           13704.997792
          November      14161.722333
          October       14031.060789
          September     11501.261702
Misc      April         13117.048621
          August        13446.105714
          December      12830.273333
          February      10797.342885
          January       10520.237037
          July          12360.065000
          June          14209.112295
          March         12439.161719
          May           11464.369403
          November      13566.254286
          October       12469.515652
          September     12088.618000
Showers   April         12004.968077
          August        10692.941509
          Decembe

# Consumer Complaints data


In [14]:
complaints = pd.read_csv('../data/complaints.csv.gz', parse_dates=['Date received'])
complaints.head()

,Date received,Product,State,ZIP code
0,2019-03-15,Debt collection,FL,334XX
1,2019-03-15,"Credit reporting, credit repair services, or o...",FL,34243
2,2019-03-15,"Credit reporting, credit repair services, or o...",TX,773XX
3,2019-03-15,"Credit reporting, credit repair services, or o...",FL,34243
4,2019-03-15,"Credit reporting, credit repair services, or o...",TX,773XX


Let's group the data by state to see which states are the 'complaining-ist'

In [15]:
complaints['State'].value_counts()[:10]

CA    170910
FL    121147
TX    103653
NY     83187
GA     64547
IL     48029
NJ     46436
PA     42652
NC     38270
OH     37248
Name: State, dtype: int64

What might be misleading about this?

In [16]:
states = pd.read_csv('../data/states.csv', index_col=0)
states.head()

,state,area,pop
Abbreviation,,,
AL,Alabama,135767,4874747.0
AK,Alaska,1723337,739795.0
AZ,Arizona,295234,7016270.0
AR,Arkansas,137732,3004279.0
CA,California,423967,39536653.0


In [17]:
complaints_by_state = complaints.groupby('State').size().rename('complaints')
complaints_by_state.head()

State
AA       24
AE      484
AK     1342
AL    13991
AP      341
Name: complaints, dtype: int64

In [18]:
merged_complaints = pd.merge(complaints_by_state, states, left_index=True, right_index=True)
merged_complaints.head()

,complaints,state,area,pop
AK,1342,Alaska,1723337,739795.0
AL,13991,Alabama,135767,4874747.0
AR,5955,Arkansas,137732,3004279.0
AZ,26525,Arizona,295234,7016270.0
CA,170910,California,423967,39536653.0


In [19]:
merged_complaints.dropna(inplace=True)

In [20]:
merged_complaints['normalized'] = merged_complaints.complaints / merged_complaints['pop']

In [21]:
merged_complaints.nlargest(15, 'normalized')

,complaints,state,area,pop,normalized
DE,6002,Delaware,6446,961939.0,0.006239
GA,64547,Georgia,153910,10429379.0,0.006189
MD,35608,Maryland,32131,6052177.0,0.005884
FL,121147,Florida,170312,20984400.0,0.005773
NJ,46436,New Jersey,22591,9005644.0,0.005156
NV,15412,Nevada,286380,2998039.0,0.005141
CA,170910,California,423967,39536653.0,0.004323
VA,36405,Virginia,110787,8470020.0,0.004298
NY,83187,New York,141297,19849399.0,0.004191
CT,13759,Connecticut,14357,3588184.0,0.003835


## Aside on concatenation

We can concatenate data as well using `pd.concat()`, which aligns indexes for us, "filling in" missing values with `nan`, `None`, or `NaT`. 

If you don't specify the `axis` optional keyword argument, Pandas will concatenate the *rows* together ("axis=0"):

In [22]:
dat = pd.concat([merged_complaints, merged_complaints.nlargest(5, 'normalized')])
dat.tail(8)

,complaints,state,area,pop,normalized
WI,12794,Wisconsin,169635,5795483.0,0.002208
WV,2959,West Virginia,62756,1815857.0,0.001630
WY,1083,Wyoming,253335,579315.0,0.001869
DE,6002,Delaware,6446,961939.0,0.006239
GA,64547,Georgia,153910,10429379.0,0.006189
MD,35608,Maryland,32131,6052177.0,0.005884
FL,121147,Florida,170312,20984400.0,0.005773
NJ,46436,New Jersey,22591,9005644.0,0.005156


In [23]:
dat.drop_duplicates(inplace=True)
dat.tail(8)

,complaints,state,area,pop,normalized
TX,103653,Texas,695662,28304596.0,0.003662
UT,7479,Utah,219882,3101833.0,0.002411
VA,36405,Virginia,110787,8470020.0,0.004298
VT,1658,Vermont,24906,623657.0,0.002659
WA,23782,Washington,184661,7405743.0,0.003211
WI,12794,Wisconsin,169635,5795483.0,0.002208
WV,2959,West Virginia,62756,1815857.0,0.001630
WY,1083,Wyoming,253335,579315.0,0.001869


We *could* have also created the `merged_complaints` `DataFrame` using concatenation (with `axis=1` to concatenate columns). 

This is equivalent to a `pd.merge()` with both `left_index=True` and `right_index=True`:

In [24]:
merged_complaints = pd.concat([complaints_by_state, states], axis=1).dropna()
merged_complaints.head()

,complaints,state,area,pop
AK,1342,Alaska,1723337.0,739795.0
AL,13991,Alabama,135767.0,4874747.0
AR,5955,Arkansas,137732.0,3004279.0
AZ,26525,Arizona,295234.0,7016270.0
CA,170910,California,423967.0,39536653.0
